In [39]:
import pickle
import sys
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd
sys.path.insert(0, './../preprocessing')

import session as ss

def topK(K,centroid,data):
    df = data.copy()
    df['distance'] = np.linalg.norm(centroid - data.loc[:, data.columns != 'label'], axis=1) 
    df = df.sort_values(by=['distance'])
    return df.head(K)

embedding_size = 50
embeddings = pd.read_csv('word2vec_embeddings/embedding_%d.csv' % embedding_size, sep='\t', header=None, dtype=float)
labels = pd.read_csv('word2vec_embeddings/labels_%d.csv' % embedding_size, sep='\t', header=None, dtype=str)

kmeans = KMeans(n_clusters=10).fit(X)

centroids = kmeans.cluster_centers_

embeddings['label'] = labels

for i, c in enumerate(centroids):
    top10 = topK(K=10, centroid=c, data=embeddings)
    print('cluster %d' % i)
    print(top10['label'])
    print()


cluster 0
7248    click on "FUNN AS (OVERVÃÂKNING HAVBRUK)"
6808                       click on "45 Abonnement"
1642                       click on "26 Abonnement"
2150                       click on "31 Abonnement"
9018                     scroll on "Page: 11469909"
841                        click on "16 Abonnement"
1177                       click on "14 Abonnement"
4814                   click on "11 Fakturakontoer"
1126             click on "Faktureringsinformasjon"
1031                       click on "24 Abonnement"
Name: label, dtype: object

cluster 1
6651                   click on "404 06 912"
4073                 click on "580000836464"
6299                         touch on "Velg"
2141        click on "BOLKAN KRANSERVICE AS"
3774                    click on "Tinfos As"
9112    click on "Voss Energi Produksjon As"
7827    click on "BUER-BERGSTRÃÂM, HENRIK"
7988                   click on "FREDRIKSEN"
2208          click on "CAMILLA HAMBO EIKEN"
2637                   clic